In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
# web scrapping
import bs4 as bs
import requests
import lxml
from functools import reduce
# matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from ipysigma import Sigma
from pyvis.network import Network
import requests
from bs4 import BeautifulSoup
from io import StringIO
from dbconnection import MySQLDatabase
from utils import getSymbols, getData, get_last_date, get_marketid_simbols
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="pandas")
sns.set_theme()

In [6]:
date = datetime.today()
today = date.strftime('%Y-%m-%d')
today

'2025-10-19'

In [7]:
db = MySQLDatabase("financialmarkets")

## Se obtienen simbolos de una url

In [8]:
url = "https://www.nasdaqtrader.com/dynamic/symdir/nasdaqlisted.txt"

In [29]:
# Descargar el contenido del archivo
response = requests.get(url)
response.raise_for_status()  # Verificar que la descarga fue exitosa

# Dividir el contenido en una lista de símbolos
data_lines = response.text.strip().split('\n')
#tickers = [line.split('|')[0] for line in tickers][1:]
# Mostrar los primeros 10 símbolos
# Separar el header
header = data_lines[0].strip().split('|')

# Separar cada línea por '|' y eliminar '\r'
rows = [line.strip().split('|') for line in data_lines[1:]]

# Crear DataFrame
df = pd.DataFrame(rows, columns=header)
print(df.shape)
# Mostrar el número total de símbolos
#print(f"\nTotal de tickers de NYSE: {len(tickers)}")

(5138, 8)


In [30]:
# generamos variables no existentes
df['GICS Sector'] = ['SD' for x in df['Symbol']]
df['GICS Sub-Industry'] = ['SD' for x in df['Symbol']]
df['Security'] = df['Security Name']
df['Headquarters Location'] = ['SD' for x in df['Symbol']]
df['CIK'] = ['SD' for x in df['Symbol']]
df['Founded'] = [9999 for x in df['Symbol']]
df['Date added'] = ['0000-00-00' for x in df['Symbol']]
df = df[['Symbol','Security','GICS Sector','GICS Sub-Industry','Headquarters Location', 'Date added','CIK','Founded']]
df.head()


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,AACB,Artius II Acquisition Inc. - Class A Ordinary ...,SD,SD,SD,0000-00-00,SD,9999
1,AACBR,Artius II Acquisition Inc. - Rights,SD,SD,SD,0000-00-00,SD,9999
2,AACBU,Artius II Acquisition Inc. - Units,SD,SD,SD,0000-00-00,SD,9999
3,AACG,ATA Creativity Global - American Depositary Sh...,SD,SD,SD,0000-00-00,SD,9999
4,AACI,Armada Acquisition Corp. II - Class A Ordinary...,SD,SD,SD,0000-00-00,SD,9999


In [ ]:
# ---------------------------
# 3️⃣ Insertar/actualizar mercados
# ---------------------------
markets = pd.DataFrame({
    'market_name': ['NASDAQ'],
    'country': ['USA'],
    'currency': ['USD']
})
markets

In [ ]:
db.insert_to_db(markets, tabla="markets", batch_size=5000)

In [11]:
# Obtener market_id
market_id = db.execute_query("SELECT * FROM markets")
market_id

✅ Conexión exitosa


,market_id,market_name,country,currency
0,1,S&P 500,USA,USD
1,2,NASDAQ,USA,USD


**Tabla 2: sectores**

In [19]:
# No se tienen información se pone como SD (sin dato)
sectors_unique = df[['GICS Sector', 'GICS Sub-Industry']].drop_duplicates()
sectors_unique.columns = ['sector_name', 'sub_industry']

In [21]:
db.insert_to_db(sectors_unique, tabla="sectors", batch_size=5000)

In [12]:
sector_map = db.execute_query("SELECT * FROM sectors")
sector_map


,sector_id,sector_name,sub_industry
0,101,Communication Services,Advertising
1,91,Communication Services,Broadcasting
2,63,Communication Services,Cable & Satellite
3,38,Communication Services,Integrated Telecommunication Services
4,82,Communication Services,Interactive Home Entertainment
...,...,...,...
124,17,Utilities,Electric Utilities
125,39,Utilities,Gas Utilities
126,8,Utilities,Independent Power Producers & Energy Traders
127,23,Utilities,Multi-Utilities


**Empresas**

In [31]:
# Agregar sector_id
df_sector = df.merge(sector_map, left_on=['GICS Sector','GICS Sub-Industry'], right_on=['sector_name','sub_industry'], how='left')

companies = df_sector[['Symbol','Security','sector_id','Headquarters Location','CIK','Founded']]
companies.columns = ['symbol','name','sector_id','headquarters','cik','founded']
#companies.loc[:,'founded'] = [str(x)[:4] for x in companies['founded']]
companies = companies.reset_index(drop=True)

In [34]:
companies.head()

,symbol,name,sector_id,headquarters,cik,founded
0,AACB,Artius II Acquisition Inc. - Class A Ordinary ...,129,SD,SD,9999
1,AACBR,Artius II Acquisition Inc. - Rights,129,SD,SD,9999
2,AACBU,Artius II Acquisition Inc. - Units,129,SD,SD,9999
3,AACG,ATA Creativity Global - American Depositary Sh...,129,SD,SD,9999
4,AACI,Armada Acquisition Corp. II - Class A Ordinary...,129,SD,SD,9999


In [35]:
db.insert_to_db(companies, tabla="companies", batch_size=1000, ignore_duplicates=True)

In [36]:
# Obtener mapping symbol -> company_id
company_map = db.execute_query("SELECT company_id, symbol FROM companies")
company_map

,company_id,symbol
0,10,A
1,1704,AACB
2,1705,AACBR
3,1706,AACBU
4,1707,AACG
...,...,...
5478,6833,ZXZZT
5479,6834,ZYBT
5480,6835,ZYME
5481,6836,ZYN


**Empresas**

In [39]:
df.columns

Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')

In [37]:
df.shape

(5138, 8)

In [40]:
market_companies_df = df[['Symbol','Date added']].merge(company_map, left_on='Symbol', right_on='symbol', how='left')
market_companies_df['market_id'] = [market_id['market_id'][1] for x in market_companies_df['Symbol']]
market_companies_df = market_companies_df[['market_id','company_id','Date added']]
market_companies_df.columns = ['market_id','company_id','date_added']
market_companies_df = market_companies_df.dropna()
market_companies_df['company_id'] = market_companies_df['company_id'].astype('int')
market_companies_df.head()


,market_id,company_id,date_added
0,2,1704,0000-00-00
1,2,1705,0000-00-00
2,2,1706,0000-00-00
3,2,1707,0000-00-00
4,2,1708,0000-00-00


In [41]:
market_companies_df.shape

(5137, 3)

In [42]:
db.insert_to_db(market_companies_df, tabla="market_companies", batch_size=1000)

## Ingresar precios

In [44]:
# mercado
mercados = ['NASDAQ']
# dataframe con compañias y id
company_map = db.execute_query("SELECT company_id, symbol FROM companies")
for mercado in mercados: 
    market_id, list_symbols = get_marketid_simbols(mercado, conn=db)
    
    for symbol in list_symbols:
        start_date = get_last_date(symbol, conn=db)
        print(f"Descargando {symbol} desde {start_date}")
        
        data = yf.download(symbol, start=start_date, progress=False, auto_adjust=False)
        
        if data.empty:
            continue
        
        data = data.reset_index()
        data.columns = [x for x,y in data.columns]
        #data['Symbol'] = [symbol for x in data['Symbol']]
        data = data.rename(columns={
            'Date': 'date',
            'Open': 'open_price',
            'Close': 'close_price',
            'High': 'high_price',
            'Low': 'low_price',
            'Volume': 'volume'
        })
        
        data['date'] = pd.to_datetime(data['date'], "%Y-%m-%d")
        # Interpolación lineal
    
        data = data.set_index('date').asfreq('D')
        #print(data.reset_index())
        
        data[['open_price','close_price','high_price','low_price','volume']] = \
            data[['open_price','close_price','high_price','low_price','volume']].interpolate(method='linear')
        data.loc[:,'symbol'] = [symbol for x in data['open_price']]
        data = data.reset_index()
        data = data.merge(company_map, left_on='symbol', right_on='symbol', how='left')
        data.loc[:,'market_id'] = [market_id for x in data['open_price']]
        # limpieza
        data_final = data[['company_id','market_id','date','open_price','close_price',
                            'high_price','low_price','volume']]
        data_final.loc[:,'company_id'] = data_final['company_id'].astype(int)
        data_final.loc[:,'market_id'] = data_final['market_id'].astype(int)
        data_final.loc[:,'date'] = pd.to_datetime(data_final['date'])
        data_final.loc[:,'open_price'] = data_final['open_price'].astype(float)
        data_final.loc[:,'close_price'] = data_final['close_price'].astype(float)
        data_final.loc[:,'high_price'] = data_final['high_price'].astype(float)
        data_final.loc[:,'low_price'] = data_final['low_price'].astype(float)
        data_final.loc[:,'volume'] = data_final['volume'].astype(int)
        # inserta datos a base 
        db.insert_to_db(data_final, tabla="stock_prices", batch_size=500)



Descargando ADBE desde 2025-10-18
Descargando AMD desde 2025-10-18
Descargando ABNB desde 2025-10-18
Descargando AKAM desde 2025-10-18
Descargando ALGN desde 2025-10-18
Descargando LNT desde 2025-10-18
Descargando GOOGL desde 2025-10-18
Descargando GOOG desde 2025-10-18
Descargando AMZN desde 2025-10-18
Descargando AEP desde 2025-10-18
Descargando AMGN desde 2025-10-18
Descargando ADI desde 2025-10-18
Descargando APA desde 2025-10-18
Descargando AAPL desde 2025-10-18
Descargando AMAT desde 2025-10-18
Descargando APP desde 2025-10-18
Descargando ACGL desde 2025-10-18
Descargando ADSK desde 2025-10-18
Descargando ADP desde 2025-10-18
Descargando AXON desde 2025-10-18
Descargando BKR desde 2025-10-18
Descargando TECH desde 2025-10-18
Descargando BIIB desde 2025-10-18
Descargando BKNG desde 2025-10-18
Descargando AVGO desde 2025-10-18
Descargando CHRW desde 2025-10-18
Descargando CDNS desde 2025-10-18
Descargando CPB desde 2025-10-18
Descargando CDW desde 2025-10-18
Descargando CHTR desde 

Failed to get ticker 'ALGT' reason: Failed to perform, curl: (28) Operation timed out after 10007 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.


Descargando ALHC desde 2000-01-01
Descargando ALIL desde 2000-01-01
Descargando ALKS desde 2000-01-01
Descargando ALKT desde 2000-01-01
Descargando ALLO desde 2000-01-01
Descargando ALLR desde 2000-01-01
Descargando ALLT desde 2000-01-01
Descargando ALLW desde 2000-01-01
Descargando ALM desde 2000-01-01
Descargando ALMS desde 2000-01-01
Descargando ALMU desde 2000-01-01
Descargando ALNT desde 2000-01-01
Descargando ALNY desde 2000-01-01
Descargando ALOT desde 2000-01-01
Descargando ALRM desde 2000-01-01
Descargando ALRS desde 2000-01-01
Descargando ALT desde 2000-01-01
Descargando ALTI desde 2000-01-01
Descargando ALTO desde 2000-01-01
Descargando ALTS desde 2000-01-01
Descargando ALTY desde 2000-01-01
Descargando ALVO desde 2000-01-01
Descargando ALVOW desde 2000-01-01
Descargando ALXO desde 2000-01-01
Descargando ALZN desde 2000-01-01
Descargando AMAL desde 2000-01-01
Descargando AMBA desde 2000-01-01
Descargando AMBR desde 2000-01-01
Descargando AMCX desde 2000-01-01
Descargando AMD

Failed to get ticker 'EDBLW' reason: Failed to perform, curl: (28) Operation timed out after 10004 milliseconds with 0 bytes received. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.


Descargando EDHL desde 2000-01-01
Descargando EDIT desde 2000-01-01
Descargando EDRY desde 2000-01-01
Descargando EDSA desde 2000-01-01
Descargando EDTK desde 2000-01-01
Descargando EDUC desde 2000-01-01
Descargando EEFT desde 2000-01-01
Descargando EEIQ desde 2000-01-01
Descargando EEMA desde 2000-01-01
Descargando EFAS desde 2000-01-01
Descargando EFOI desde 2000-01-01
Descargando EFRA desde 2000-01-01
Descargando EFSC desde 2000-01-01
Descargando EFSCP desde 2000-01-01
Descargando EFSI desde 2000-01-01
Descargando EFTY desde 2000-01-01
Descargando EGAN desde 2000-01-01
Descargando EGBN desde 2000-01-01
Descargando EGGQ desde 2000-01-01
Descargando EGHA desde 2000-01-01
Descargando EGHAR desde 2000-01-01
Descargando EGHAU desde 2000-01-01
Descargando EGHT desde 2000-01-01
Descargando EH desde 2000-01-01
Descargando EHGO desde 2000-01-01
Descargando EHLD desde 2000-01-01
Descargando EHLS desde 2000-01-01
Descargando EHTH desde 2000-01-01
Descargando EJH desde 2000-01-01
Descargando EK

...

Descargando ZCMD desde 2000-01-01
Descargando ZCZZT desde 2000-01-01



1 Failed download:
['ZCZZT']: YFTzMissingError('possibly delisted; no timezone found')


Descargando ZD desde 2000-01-01
Descargando ZDAI desde 2000-01-01
Descargando ZENA desde 2000-01-01
Descargando ZENV desde 2000-01-01
Descargando ZEO desde 2000-01-01
Descargando ZEOWW desde 2000-01-01
Descargando ZEUS desde 2000-01-01
Descargando ZFZZT desde 2000-01-01
Descargando ZG desde 2000-01-01
Descargando ZGM desde 2000-01-01
Descargando ZIMV desde 2000-01-01
Descargando ZION desde 2000-01-01
Descargando ZIONP desde 2000-01-01
Descargando ZIPP desde 2000-01-01
Descargando ZJK desde 2000-01-01
Descargando ZJYL desde 2000-01-01
Descargando ZJZZT desde 2000-01-01
Descargando ZKIN desde 2000-01-01
Descargando ZLAB desde 2000-01-01
Descargando ZM desde 2000-01-01
Descargando ZMUN desde 2000-01-01
Descargando ZNB desde 2000-01-01
Descargando ZNTL desde 2000-01-01
Descargando ZOOZ desde 2000-01-01
Descargando ZOOZW desde 2000-01-01
Descargando ZS desde 2000-01-01
Descargando ZSPC desde 2000-01-01
Descargando ZTEK desde 2000-01-01
Descargando ZTEN desde 2000-01-01
Descargando ZTOP desd


1 Failed download:
['ZXYZ.A']: YFTzMissingError('possibly delisted; no timezone found')


Descargando ZXZZT desde 2000-01-01
Descargando ZYBT desde 2000-01-01
Descargando ZYME desde 2000-01-01
Descargando ZYN desde 2000-01-01
Descargando ZYXI desde 2000-01-01


In [ ]:
# Example tickers (use full S&P 500 list if you want)
#tickers = stocks2

# Download adjusted close prices
#prices = yf.download(tickers, start='2020-01-01', end='2025-01-01')['Close']

#prices = prices.fillna(method='bfill')
# Calculate daily returns
returns = data.pct_change().dropna()

# Compute correlation matrix
correlation_matrix = returns.corr()

# Set correlation threshold
threshold = 0.7

# Create graph
G = nx.Graph()

# Add nodes
for ticker in correlation_matrix.columns:
    G.add_node(ticker)

# Add edges with correlation above threshold
for i, stock1 in enumerate(correlation_matrix.columns):
    for j, stock2 in enumerate(correlation_matrix.columns):
        if i < j:
            corr = correlation_matrix.iloc[i, j]
            if abs(corr) >= threshold:
                G.add_edge(stock1, stock2, weight=corr)

# Display with ipysigma
#Sigma(G)
# Initialize pyvis network
#net = Network(notebook=True)
#net.from_nx(G)

# Save and show the graph
#net.show("graph.html")
Sigma.write_html(
    G,
    './dataset.html',
    fullscreen=True,
    node_metrics=['louvain'],
    node_color='louvain',
    node_size_range=(3, 15),
    max_categorical_colors=30,
    default_edge_type='curve',
    node_border_color_from='node',
    default_node_label_size=14,
    node_size=G.degree
)


In [ ]:
from IPython.display import IFrame
IFrame('graph.html', width=800, height=600)